In [ ]:
# This notebook will tack the coron-specific filter boresight offsets from a CSV file
# into the FITS distortion reference file
# 
# From here it can be called by miricoord, pulled out to pipeline ref files, etc.
#
# Offsets will be ZERO for filters in which it hasn't been measured for the coronagraphs,
# including for F770W, which will ensure that we can call miricoord in the usual manner
# and have it work as expected since we're tying F770W to each coron filter

In [ ]:
from astropy.io import fits
import numpy as np
from astropy.io import ascii
import miricoord.imager.mirim_tools as mt

In [ ]:
distfile=mt.get_fitsreffile()
disthdus=fits.open(distfile)
borehdu=disthdus['Boresight offsets']

In [ ]:
fits.info(distfile)

In [ ]:
borehdu.data

In [ ]:
# Make a copy of the boresight HDU for MASK1065, MASK1140, MASK1550, MASKLYOT
borehdu1065=borehdu.copy()
borehdu1065.header['EXTNAME']='BoresightCORON1065'

borehdu1140=borehdu.copy()
borehdu1140.header['EXTNAME']='BoresightCORON1140'

borehdu1550=borehdu.copy()
borehdu1550.header['EXTNAME']='BoresightCORON1550'

borehdulyot=borehdu.copy()
borehdulyot.header['EXTNAME']='BoresightCORONLYOT'

nfilter=len(borehdu.data)
print('nfilter = ',nfilter)

In [ ]:
# Read the updated coron offsets csv
coron=ascii.read('coronagraph_boresight_offsets-pixels.csv')

In [ ]:
coron

In [ ]:
# Populate information in the borehdu1065 HDU
for ii in range(0,nfilter):
    indx=np.where((coron['subarray'] == 'MASK1065')&(coron['filter'] == borehdu1065.data['FILTER'][ii]))[0]
    if (len(indx) == 1):
        borehdu1065.data['COL_OFFSET'][ii]=coron[indx]['dx']
        borehdu1065.data['ROW_OFFSET'][ii]=coron[indx]['dy']
    else:
        borehdu1065.data['COL_OFFSET'][ii]=0
        borehdu1065.data['ROW_OFFSET'][ii]=0

In [ ]:
# Populate information in the borehdu1140 HDU
for ii in range(0,nfilter):
    indx=np.where((coron['subarray'] == 'MASK1140')&(coron['filter'] == borehdu1140.data['FILTER'][ii]))[0]
    if (len(indx) == 1):
        borehdu1140.data['COL_OFFSET'][ii]=coron[indx]['dx']
        borehdu1140.data['ROW_OFFSET'][ii]=coron[indx]['dy']
    else:
        borehdu1140.data['COL_OFFSET'][ii]=0
        borehdu1140.data['ROW_OFFSET'][ii]=0

In [ ]:
# Populate information in the borehdu1550 HDU
for ii in range(0,nfilter):
    indx=np.where((coron['subarray'] == 'MASK1550')&(coron['filter'] == borehdu1550.data['FILTER'][ii]))[0]
    if (len(indx) == 1):
        borehdu1550.data['COL_OFFSET'][ii]=coron[indx]['dx']
        borehdu1550.data['ROW_OFFSET'][ii]=coron[indx]['dy']
    else:
        borehdu1550.data['COL_OFFSET'][ii]=0
        borehdu1550.data['ROW_OFFSET'][ii]=0

In [ ]:
# Populate information in the borehdulyot HDU
for ii in range(0,nfilter):
    indx=np.where((coron['subarray'] == 'MASKLYOT')&(coron['filter'] == borehdulyot.data['FILTER'][ii]))[0]
    if (len(indx) == 1):
        borehdulyot.data['COL_OFFSET'][ii]=coron[indx]['dx']
        borehdulyot.data['ROW_OFFSET'][ii]=coron[indx]['dy']
    else:
        borehdulyot.data['COL_OFFSET'][ii]=0
        borehdulyot.data['ROW_OFFSET'][ii]=0

In [ ]:
# Append new HDUs to the distortion file
newhdus=disthdus.copy()

for key, borehdu in [('BoresightCORON1065', borehdu1065),
                 ('BoresightCORON1140', borehdu1140),
                 ('BoresightCORON1550', borehdu1550),
                 ('BoresightCORONLYOT', borehdulyot)
                 ]:
    try:
        newhdus[key] = borehdu
    except KeyError:
        newhdus.append(borehdu)
        
newhdus.info()

In [ ]:
# Update header info
newhdus[0].header['FILENAME']='MIRI_FM_MIRIMAGE_DISTORTION_SS.06.00.fits'
newhdus[0].header['VERSION']='SS.06.00'
newhdus[0].header['HISTORY']='SS.06.00 updates Coron-specific boresight tables'
newhdus[0].header

In [ ]:
new_path = '../../imager/makeboresight/MIRI_FM_MIRIMAGE_DISTORTION_SS.06.00.fits'
newhdus.writeto(new_path, overwrite=True)

In [ ]:
borehdu1550.data

In [ ]:
fits.getdata(new_path, 20)